In [1]:
import pandas as pd
import pdfplumber as pdfp
import numpy as np
import os
import re


### Updated

In [2]:
def drop_unneeded(item):
    if '(' in item:
        return item[:item.find('(')]
    else:
        return item
    
def make_list(pdf):
    company_list = []
    page_num = len(pdf.pages[:])
    com = re.compile(r"[LTM\n]?\(\$Ms\)(.*)|Investment\sLTM(.*)|\(?\$\sin\s[mM]illions\)?\s(.*)",re.DOTALL)
    for i in range(page_num):
        if re.search(com,pdf.pages[i].extract_text()):
            if pdf.pages[i].extract_text().split('\n')[0][0].isdigit():
                company_list.append('a_'+pdf.pages[i].extract_text().split('\n')[0].replace(' ','_').replace(',','_'))
            else:
                company_list.append(pdf.pages[i].extract_text().split("\n")[0].replace(' ','_').replace(',','_'))
    return company_list


def make_df(pdf,company_name):
    page_num = len(pdf.pages[:])
    com = re.compile(r"[LTM\n]?\(\$Ms\)(.*)|Investment\sLTM(.*)|\(?\$\sin\s[mM]illions\)?\s(.*)",re.DOTALL)
           

    tt = company_name.split('\n')
    if tt[0] == '':
        del tt[0]
    first_value = re.compile(r"(^\D*[a-zA-Z\s]\(?\d?\)?)\s",re.DOTALL)
    first_value_with_D = re.compile(r"(^\D*\D\(?\d?\)?)\sDeal",re.DOTALL)

    df = pd.DataFrame()
    for i in tt:
        try: 
    #print(re.search(second_value,i).group(1))
    #print(re.findall(first_value,i))
            if 'Deal' in re.search(first_value,i).group(1):
                first = re.search(first_value_with_D,i).group(1)
                second = i.strip(first).split()
                in_df = pd.DataFrame(dict([(first,second)]),index = pd.Series(tt[0].split()).map(lambda x:str(x)).map(drop_unneeded))
                df = pd.concat([df,in_df],axis=1)
            elif 'NA' in re.findall(first_value,i)[0] or 'NM' in re.findall(first_value,i[1:])[0]:
                first = re.search(first_value,i.replace('NA','0NA').replace('NM','0NA')).group(1)
                second = i.strip(first).split()
                in_df = pd.DataFrame(dict([(first,second)]),index = pd.Series(tt[0].split()).map(lambda x:str(x)).map(drop_unneeded))
                df = pd.concat([df,in_df],axis=1)            
            else:
                first = re.search(first_value,i).group(1)
                second = i.strip(first).split()
                in_df = pd.DataFrame(dict([(first,second)]),index = pd.Series(tt[0].split()).map(lambda x:str(x)).map(drop_unneeded))
                df = pd.concat([df,in_df],axis=1)
        except:
            continue

    df = df.replace('0NA','N/A')
    lst = []
    lstsi = []
    for i in range(1,11):
        lst.append('('+str(i)+')')
        lstsi.append(str(i)+')')
    for i in lst:
        for j in lstsi:
            try:
                df.drop(i,axis=1,inplace = True)
            except:
                try:
                    df.drop(j,axis=1,inplace = True)
                except:
                    continue

    return df

def download_csv():
    for name in os.listdir('./'):
        if '.pdf' in name:
            new_folder = './'+name[:-4]
            if not os.path.exists(new_folder):
                os.makedirs(new_folder)
            pdf_path = './'+name
            pdf = pdfp.open(pdf_path)
            com1 = re.compile(r"[LTM\n]?\(\$Ms\)(.*)",re.DOTALL)
            com2 = re.compile(r"Investment\sLTM\n(.*)",re.DOTALL)
            com3 = re.compile(r"\(?\$\sin\s[mM]illions\)?\s(.*)",re.DOTALL)
            com4 = re.compile(r"LTM\n(\d.*)",re.DOTALL)
            com5 = re.compile(r"in\s000s\s(.*)",re.DOTALL)
            page_num = len(pdf.pages[:])
            for i in range(page_num):
                for j in [com1,com2,com3,com4,com5]:
                    if re.search(j,pdf.pages[i].extract_text()):
                        if pdf.pages[i].extract_text().split('\n')[0][0].isdigit():
                            locals()['a_'+pdf.pages[i].extract_text().split('\n')[0].replace(' ','_')] = re.search(j,pdf.pages[i].extract_text()).group(1)
                        else:
                            locals()[pdf.pages[i].extract_text().split("\n")[0].replace(' ','_')] = re.search(j,pdf.pages[i].extract_text()).group(1)

            for i in make_list(pdf):
                try:
                    make_df(pdf,eval(i)).to_csv(new_folder+'/%s.csv'%i)
                except:
                    continue
download_csv()

### Original

In [10]:
def drop_unneeded(item):
    if '(' in item:
        return item[:item.find('(')]
    else:
        return item

def make_list(pdf):
    company_list = []
    page_num = len(pdf.pages[:])
    com = re.compile(r"[LTM\n]?\(\$Ms\)(.*)|Investment\sLTM(.*)|\(?\$\sin\s[mM]illions\)?\s(.*)",re.DOTALL)
    for i in range(page_num):
        if re.search(com,pdf.pages[i].extract_text()):
            if pdf.pages[i].extract_text().split('\n')[1][0].isdigit():
                company_list.append('a_'+pdf.pages[i].extract_text().split('\n')[1].replace(' ','_').replace(',','_'))
            else:
                company_list.append(pdf.pages[i].extract_text().split("\n")[1].replace(' ','_').replace(',','_'))
    return company_list


def make_df(pdf,company_name):
    page_num = len(pdf.pages[:])
    com = re.compile(r"[LTM\n]?\(\$Ms\)(.*)|Investment\sLTM(.*)|\(?\$\sin\s[mM]illions\)?\s(.*)",re.DOTALL)
           

    tt = company_name.split('\n')
    if tt[0] == '':
        del tt[0]
    first_value = re.compile(r"(^\D*\w\(?\d?\)?)\s")
    second_value = re.compile(r"^\D*\w\(?\d?\)?\s(.*)")
    first_value_with_D = re.compile(r"(^\D*\D\(?\d?\)?)\sDeal")
    second_value_with_D = re.compile(r"^\D*\D\(?\d?\)?\s(Deal.*)")
    df = pd.DataFrame()
    for i in tt:
        try: 
    #print(re.search(second_value,i).group(1))
    #print(re.findall(first_value,i))
            if 'Deal' in re.findall(first_value,i)[0]:
                in_df = pd.DataFrame(dict([(re.findall(first_value_with_D,i)[0],re.search(second_value_with_D,i).group(1).split())]))
                df = pd.concat([df,in_df],axis=1)
            elif 'NA' in re.findall(first_value,i)[0] or 'NM' in re.findall(first_value,i)[0]:
                in_df = pd.DataFrame(dict([(re.findall(first_value,i.replace('NA','0NA').replace('NM','0NA'))[0],re.search(second_value,i.replace('NA','0NA').replace('NM','0NA')).group(1).split())]))
                df = pd.concat([df,in_df],axis=1)            
            else:    
                in_df = pd.DataFrame(dict([(re.findall(first_value,i)[0],re.search(second_value,i).group(1).split())]))
                df = pd.concat([df,in_df],axis=1)
        except:
            continue
    df = df.dropna(thresh = len(df.columns)*0.6)
    df = df.dropna(thresh = len(df)*0.5,axis=1)
    df = df.set_index(df.columns[0])
    df = df.replace('0NA','N/A')
    lst = []
    lstsi = []
    for i in range(1,11):
        lst.append('('+str(i)+')')
        lstsi.append(str(i)+')')
    for i in range(len(lst)):
        try:
            df.drop(lst[i],axis=1,inplace = True)
        except:
            try:
                df.drop(lstsi[i],axis = 1, inplace = True)
            except:
                continue
          

    return df

def download_csv():
    for name in os.listdir('./'):
        if '.pdf' in name:
            new_folder = './'+name[:-4]
            os.makedirs(new_folder)
            pdf_path = './'+name
            pdf = pdfp.open(pdf_path)
            com = re.compile(r"[LTM\n]?\(\$Ms\)(.*)|Investment\sLTM(.*)|\(?\$\sin\s[mM]illions\)?\s(.*)",re.DOTALL)
            page_num = len(pdf.pages[:])
            for i in range(page_num):
                    if re.search(com,pdf.pages[i].extract_text()):
                        if pdf.pages[i].extract_text().split('\n')[1][0].isdigit():
                            locals()['a_'+pdf.pages[i].extract_text().split('\n')[1].replace(' ','_')] = re.search(com,pdf.pages[i].extract_text()).group()
                        else:
                            locals()[pdf.pages[i].extract_text().split("\n")[1].replace(' ','_')] = re.search(com,pdf.pages[i].extract_text()).group()
            for i in make_list(pdf):
                try:
                    make_df(pdf,eval(i)).to_csv(new_folder+'/%s.csv'%i)
                except:
                    continue
download_csv()